In [1]:
import torch
from torch import nn
from torch.nn.functional import cross_entropy, softmax, binary_cross_entropy_with_logits

## Debug Focal loss

In [2]:
%run focal.py

In [3]:
alpha = 0.75
gamma = 2

### Multilabel case

In [108]:
outputs = torch.tensor([[1, 1, 0, 0, 0, 1],
                        [0, 0, 1, 0, 0, 1],
                       ]).float()
targets = torch.tensor([[1, 1, 1, 0, 1, 0],
                        [0, 1, 0, 0, 1, 0],
                       ]).float()
outputs

tensor([[1., 1., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1.]])

In [116]:
focal = FocalLoss(alpha=0.87, reduction='none')
focal(outputs, targets)

tensor([[0.8700, 0.8700, 0.8700, 0.1300, 0.8700, 0.1300],
        [0.1300, 0.8700, 0.1300, 0.1300, 0.8700, 0.1300]])
tensor([[0.0723, 0.0723, 0.2500, 0.2500, 0.2500, 0.5344],
        [0.2500, 0.2500, 0.5344, 0.2500, 0.2500, 0.5344]])
tensor([[0.3133, 0.3133, 0.6931, 0.6931, 0.6931, 1.3133],
        [0.6931, 0.6931, 1.3133, 0.6931, 0.6931, 1.3133]])


tensor([[0.0197, 0.0197, 0.1508, 0.0225, 0.1508, 0.0912],
        [0.0225, 0.1508, 0.0912, 0.0225, 0.1508, 0.0912]])

### Multiclass case

In [38]:
outputs = torch.tensor([[0, 1, 0],
                        [-3, 1, 2],
                        [1, 11.1, 0.5],
                       ]).float()
targets = torch.tensor([1, 2, 0]).long()

In [39]:
focal = FocalLoss(alpha=0.9, gamma=2, multilabel=False, reduction='none')
focal(outputs, targets)

tensor([0.0892, 0.0213, 9.0893])